In [7]:
### 주피터노트북 이미지 확인용 함수

def plt_imshow(title='image', img=None, figsize=(8 ,5)):
    plt.figure(figsize=figsize)
 
    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []
 
            for i in range(len(img)):
                titles.append(title)
 
        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
 
            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])
 
        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

In [8]:
import pytesseract
import numpy as np
import cv2
import matplotlib.pyplot as plt
# import imutils
# from imutils.perspective import four_point_transform
# from imutils.contours import sort_contours

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

config = ('-l kor+eng --oem 3 --psm 4') 
test_img="/Users/kjh001/Desktop/study/ai_study/eggo_test.png"

img=cv2.imread(test_img)
# cv2.imshow("ori",img) #이미지 띄우기

def gray_scale(image):
    # img_gray=cv2.cvtColor(img, cv2.COLOR_BAYER_BG2GRAY) // 왜 오류나는지 모르겠음
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# def thresholding(image): # 이미지 임계처리 (이진화)
#     # threshold(src,thresh,maxval,type) //thresh:임계값, maxval:임계값을 넘었을 때 적용할 value, type:thresholding type
#     # thresholding type: cv2.THRESH_BINARY/cv2.THRESH_BINARY_INV/cv2.THRESH_TRUN/,cv2.THRESH_TOZERO/cv2.THRESH_TOZERO_INV
#     result=cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
#     return result

# def remove_noise(image): # 노이즈제거 (커널 사이즈는 홀수로)
#     return cv2.medianBlur(image,5)

# # Erode - Dilate = Opening 연산: 주로 작은 노이즈 제거
# # Dilate - Erode = Closing 연산: 한 객체 추출했을 때 두 개 이상의 작은 부분으로 나올 경우 큰 객체로 합칠 때 사용
# def dilate(image): # 필터 내부의 가장 밝은(높은) 값으로 변환(or)
#     kernel= np.ones((5,5),np.uint8)
#     return cv2.dilate(image,kernel, iterations=1)

# def erode(image): # 필터 내부의 가장 낮은(어두운) 값으로 변환(and)
#     kernel = np.ones((5,5),np.uint8)
#     return cv2.erode(image, kernel, iterations=1)

# # morphologyEx: opening, closing 수행 함수
# def opening(image):
#     kernel = np.ones((5,5),np.uint8)
#     return cv2.morphologyEx(image,cv2.MORPH_OPEN,kernel)

# def canny(image): # 이미지 엣지 리턴
#     return cv2.Canny(image,100,200) # 숫자 작게할수록 자세히

def deskew(image): # 숫자 이미지를 읽어 기울어진 이미지 보정
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    thresh = cv2.threshold(gray, 0, 255,
        cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h),
        flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)    
    return rotated

# def match_template(image, template): # 이미지에서 템플릿과 일치하는 부분 찾음
#     return cv2.matchTemplate(image,template, cv2.TM_CCOEFF_NORMED)

def filter2D(image): # 이미지 선명하게
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(image, 0, kernel)

deskew = deskew(img)
gray = gray_scale(deskew)

print(img.shape)

(1336, 916, 3)


# 이미지 속 텍스트 인식 구현

In [9]:
### 이미지 내 관심영역 추출과 텍스트 추출

# x,y 반대로
x_indivi=770; y_indivi=20; w_indivi=450; h_indivi=340    # roi 좌표(개별관리비)
x_date=480; y_date=450; w_date=220; h_date=270     # roi 좌표(날짜 정보)
x_price=480; y_price=670; w_price=230; h_price=270     # roi 좌표(이번 달 총금액)


roi1 = gray[x_indivi:x_indivi+h_indivi , y_indivi:y_indivi+w_indivi]  # roi 지정
roi2 = gray[x_date:x_date+h_date , y_date:y_date+w_date]  # roi 지정
roi3 = gray[x_price:x_price+h_price, y_price:y_price+w_price]  # roi 지정

cv2.rectangle(roi1, (0,0), (w_indivi-1, h_indivi-1), (0,0,255) ,2) # roi 전체에 사각형 그리기
cv2.rectangle(roi2, (0,0), (w_date-1, h_date-1), (0,0,255) ,2) # roi 전체에 사각형 그리기
cv2.rectangle(roi3, (0,0), (w_price-1, h_price-1), (0,0,255) ,2) # roi 전체에 사각형 그리기

text_indivi=pytesseract.image_to_string(roi1,config=config)
text_date=pytesseract.image_to_string(roi2,config=config)
text_price=pytesseract.image_to_string(roi3,config=config)


## 확인용

print("-----개별관리비-----")
print(text_indivi)
print("-----날짜정보-----")
print(text_date)
print("-----총금액-----")
print(text_price)

# cv2.imshow("image", gray)
# key = cv2.waitKey(0)
# print(key)
# cv2.destroyAllWindows()

-----개별관리비-----
공용소계         91,590     91,910     -320

전기 331 kw        41,280    40,610    670
Weae           2,500      2,500
케이블 TV          4,400      4,400
주차비
수도 23 03           25,740      23,050    2,690
일자리청소차         -140      -140        {
일자리경비차         -200      -340 140
관리비 과남

AWA        73,580 70,080 3,500


-----날짜정보-----
2022년 02일분

112동 306호
기2코드 4060-161
[BRAS — 0112-0306-8
월분 2022-02

데카드 자동이체 *
삼각

-----총금액-----
: 남입수증 Ea)

165,170

그린타운 판리사무소 귀중



# 인식된 텍스트 정리
# 고지서 등록하기 & 고지서 조회하기
### 조회된 고지서 내용
고지서 딕셔너리 - 날짜: 전기료, 수도료, 공용, 총요금

In [10]:
gojiseo={}

def _gojiseo():
### 날짜 정보
    start_date=text_date.find('월분')+2
    date_l=text_date[start_date:start_date+10].strip().strip('、').strip(',').strip('.').split("-")
    date_this=date_l[0]+date_l[1]

### 전기
    start_use_e = text_indivi.find('전기')+2
    elec_use = int(text_indivi[start_use_e:start_use_e+4].strip())
    start_price_e = text_indivi.find('kw')+2
    elec_price = text_indivi[start_price_e:start_price_e+18].strip()
    elec_price = int(elec_price.replace(",",'').strip())

### 수도
    start_use_w = text_indivi.find('수도')+2
    water_use = int(text_indivi[start_use_w:start_use_w+3].strip())
    start_price_w = text_indivi.find('수도')+15
    water_price = text_indivi[start_price_w:start_price_w+15].strip()
    water_price = int(water_price.replace(",",'').strip())

### 총금액 --> !!위치로 따온거라 수정해야 할수도 있음!!
    splits=text_price.splitlines()
    price_this=splits[2]
    total_price = int(price_this.replace(",",'').strip())

### 공용관리비 정보
    start_use_public = text_indivi.find('공용소계')+10
    public_price = text_indivi[start_use_public:start_use_public+10].strip()
    public_price = int(public_price.replace(",",'').strip())

# print("날짜 정보(yyyymm): ", date_this,'\n')
# print("전기 사용량:", elec_use,"kw")
# print("전기 요금:", elec_price,'원\n')
# print("수도 사용량:", water_use,"m3")
# print("수도 요금:", water_price,'원\n')
# print("공용 관리비: ", public_price,'원\n')
# print("총 요금:", total_price,"원")

    gojiseo[date_this]=[elec_use, elec_price, water_use, water_price, public_price, total_price]
    
    return gojiseo

_gojiseo()

{'202202': [331, 41280, 23, 25740, 91590, 165170]}

# 메인
### 고지서 정보 그래프

In [11]:
# def electric_or_water(x):
#     plt.figure(figsize=(10,5)) 
#     if selectElectric==True:        
#         plt.plot(x,test_elec_price[-len(x):],marker='o')
#     elif selectWater==True:
#         plt.plot(x,test_wat_price[-len(x):],marker='o')
#     elif selectTotal==True:
#         plt.plot(x,test_total_price[-len(x):],marker='o')
        
# select_3m_6m_12m(select3m,select6m,select12m)

# 자세히 보기 & 고지서 조회하기
### 이번 달 요금 원 그래프 

In [12]:
import matplotlib.pyplot as plt
import pandas as pd

def pie_graph():
    etc_indivi_price = total_price-(elec_price + water_price + public_price)
    
    return elec_price, water_prece, etc_indivi_price, public_price, total_price
#     graph_series = pd.Series([elec_price, wat_price, etc_indivi_price, public_price], index = ['electric', 'water', 'individual etc', 'public'])
#     colors = pd.Series(['gold', 'skyblue', 'silver', 'pink'], index = graph_series.index)
#     plt.pie(graph_series, labels = graph_series.index, autopct = '%.1f%%', colors=colors)
#     plt.legend(loc = (1.1,0.35))
#     plt.show()

# print("합계:",total_price,"원")

# 고지서 예측하기
### 사용자 데이터를 이용한 다음달 납부금액 예측 & 전년 동월 대비 상승률 표시

In [27]:
### 다음 달 금액 예측하기 (테스트용 데이터)

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 고지서에서 읽은 사용자의 사용량 데이터라고 가정(13개)
# goji_month_list, goji_elect_price, goji_water_price로 변경

test_elect_price=[35000,38010,28150,28800,27160,39330,37470,51389,42140,34740,36410,33390,40610] # 41280
test_water_price=[24990,28430,29780,24390,25740,25740,29780,28430,28430,32470,27090,28430,23050] # 2574
test_total_price=[173560,185120,171060,165720,165560,175850,157110,167680,159940,161840,153840,157420,161990] # 165170

def rbf_predict():
    months=[]
    j=1
    for j in range(len(test_elect_price)):
        months.append([int(j+1)])
        j=j+1

    x_train_t, x_test_t, y_train_t ,y_test_t = train_test_split(months,test_total_price,test_size=0.2, random_state=777)

    rbf_svr_t1 = SVR(kernel='rbf', C=20000, gamma=0.3)
    rbf_svr_t1.fit(x_train_t, y_train_t)

    
### 다음 달 금액 예측
    nextMonth = [[len(months)+1]]
    total_predict = rbf_svr_t1.predict(nextMonth)
    total_predict = int(total_predict[0])

#     print('예상 총금액: ', round(total_predict[0]),",",round(total_predict[1]),",",round(total_predict[2]) )
#     print("진짜 총금액:", test_total_price[x_test_t[0][0]],",",test_total_price[x_test_t[1][0]],",",test_total_price[x_test_t[2][0]],"\n")

#     print("총금액 훈련 정확도: ", round(rbf_svr_t1.score(x_train_t, y_train_t),2)) # 74.29
#     print("총금액 테스트 정확도: ", r2_score(y_test_t, total_predict)) # 74.11

    
### 전년 동월 금액 대비 상승률 표시
    rising_rate=(rbf_svr_t1.predict(nextMonth)/test_total_price[-11])*100
    comparison=round(rising_rate[0],2)
#     print("\n 작년 동월 금액의 약",comparison,"% 입니다.")
    return total_predict, comparison

user_data_predict=rbf_predict()
print(user_data_predict)

# score 결정계수
# 회귀모델이 주어진 자료에 얼마나 적합한지를 평가하는 지표
# y의 변동량대비 모델 예측값의 변동량을 의미함
# 0~1의 값을 가지며, 상관관계가 높을수록 1에 가까워짐
# r2=0.3인 경우 약 30% 정도의 설명력을 가진다 라고 해석할 수 있음
# 일반적으로 Bio는 0.95, 공학은 0.7, 사회과학은 0.3 정도를 기준으로 한다고 함

(166957, 97.6)


# Eggo AI의 충고
### api 사용해 단지 평균 전기료, 수도료 가져오기
(시간남으면 추가)

# 아파트 평균 수도료+전기료와 사용자의 수도료+전기료 비교 그래프 (eggo AI의 충고)

In [16]:
import requests
import xmltodict
import json
from datetime import datetime
from dateutil.relativedelta import *
from sklearn.svm import SVR
import pandas as pd
# from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

def danji_predict(sido_code, apt_name):
    ## sido_code: {'대구':'27','인천':'28','광주':'29','대전':'30','울산':'31','경기':'41','강원':'42','충북':'43','충남':'44','전북':'45','전남':'46','경북':'47','경남':'48','제주':'50'}
    ### 단지코드 찾기
    sido,name=sido_code, apt_name
    def apt_danji(sido,name):
        service_dkey = 'X3P5kZCUe75RwQN0GH80YdI51JbHHwlOWkQLJiRyFIbgn/RiaVJmjOX/yFbt50x4rAhtPGt4Jlya+rIJlUefaA=='
        url = 'http://apis.data.go.kr/1613000/AptListService2/getSidoAptList'
        param = {'serviceKey':service_dkey, 'sidoCode':sido, 'pageNo':1, 'numOfRows':'1'}
        r = requests.get(url,params=param)
    
        xmlData = r.content.decode('utf-8')
        parseData = xmltodict.parse(xmlData)
        jData = json.loads(json.dumps(parseData))
    
        rows=jData['response']['body']['totalCount']
    
        pages = int(int(rows)/100)+1    
        row_cnt=1
        for i in range(1, pages+1):
            params={'serviceKey':service_dkey, 'sidoCode':sido, 'pageNo':i, 'numOfRows':'100'}
        
            response = requests.get(url, params=params)
            xml_data = response.content.decode('utf-8')
            parse_data = xmltodict.parse(xml_data)
        
            ord_data = parse_data['response']['body']['items']['item']
        
            try:
                jdata = json.loads(json.dumps(ord_data))
                input_danji_name=name
            
                for code_data in jdata:
                    danji_code=code_data['kaptCode']
                    danji_name=code_data['kaptName']
                    row_cnt += 1
                    if danji_name == input_danji_name:
                        # print(code_data)
                        return danji_code
                
            except Exception as e:
                print(f'Error: {e}\n')
                break

    # 단지별 전기, 수도 총 사용량을 세대수로 나눠 평균 사용양 알아내기
    ### api로 단지별 세대수 불러오기
    def danji_people(code):
        url = 'http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo'
        params ={'serviceKey':'X3P5kZCUe75RwQN0GH80YdI51JbHHwlOWkQLJiRyFIbgn/RiaVJmjOX/yFbt50x4rAhtPGt4Jlya+rIJlUefaA==',
         'kaptCode' : code}
        response = requests.get(url, params=params)
        xml_data = response.content.decode('utf-8')
        parse_data = xmltodict.parse(xml_data)
        ord_data = parse_data['response']['body']
        jData = json.loads(json.dumps(ord_data))
        jData = jData['item']
        return jData['kaptdaCnt']

    ### api로 단지별 전기 사용량 불러오기
    def danji_elect(code,date):
        url = 'http://apis.data.go.kr/1611000/AptIndvdlzManageCostService/getHsmpElectricityCostInfo'
        params ={'serviceKey':'X3P5kZCUe75RwQN0GH80YdI51JbHHwlOWkQLJiRyFIbgn/RiaVJmjOX/yFbt50x4rAhtPGt4Jlya+rIJlUefaA==',
         'kaptCode' : code, 'searchDate' : str(date) }
        response = requests.get(url, params=params)
        xml_data = response.content.decode('utf-8')
        parse_data = xmltodict.parse(xml_data)
        ord_data = parse_data['response']['body']
        jData = json.loads(json.dumps(ord_data))
        jData = jData['item']
        return jData['electP']

    ### api로 단지별 수도 사용량 불러오기
    def danji_water(code,date):
        url = 'http://apis.data.go.kr/1611000/AptIndvdlzManageCostService/getHsmpWaterCostInfo'
        params ={'serviceKey':'X3P5kZCUe75RwQN0GH80YdI51JbHHwlOWkQLJiRyFIbgn/RiaVJmjOX/yFbt50x4rAhtPGt4Jlya+rIJlUefaA==',
         'kaptCode' : code, 'searchDate' : str(date) }
        response = requests.get(url, params=params)
        xml_data = response.content.decode('utf-8')
        parse_data = xmltodict.parse(xml_data)
        ord_data = parse_data['response']['body']
        jData = json.loads(json.dumps(ord_data))
        jData = jData['item']
        return jData['waterCoolP']

    danji_price_dict={}
    danji_elect_price_list=[]
    danji_water_price_list=[]

    months_list=[]
    present_date = datetime.today().strftime("%Y%m")

    # 단지 평균 전기료, 수도료 조회 (13개월 16-3)
    for i in range(2,27): # 현재 달로부터 3개월 전부터 조회가능
        ago_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-(28-i))).strftime("%Y%m")
        danji_code = apt_danji(sido_code,apt_name)
        
        electP = danji_elect(danji_code, ago_month) # 우리 단지 총액
        waterP = danji_water(danji_code, ago_month) # 우리 단지 총액

        danji_elect_price = round(int(electP)/int(danji_people(danji_code))) # 우리 단지 평균
        danji_water_price = round(int(waterP)/int(danji_people(danji_code)))
        danji_elect_price_list.append(danji_elect_price)
        danji_water_price_list.append(danji_water_price)
    
        danji_price_dict[ago_month]= [danji_elect_price, danji_water_price] # 날짜 : 전기료, 수도료
        months_list.append(ago_month)
    
    
    
    test_elect_price=[35000,38010,28150,28800,27160,39330,37470,51389,42140,34740,36410,33390,40610] # 41280
    test_water_price=[24990,28430,29780,24390,25740,25740,29780,28430,28430,32470,27090,28430,23050] # 2574
    danji_ew=[]
    user_ew=[]
    user_elect_price=test_elect_price
    user_water_price=test_water_price

    months=[]
    j=1
    for j in range(len(danji_elect_price_list)):
        months.append([int(j+1)])
        j=j+1
    
    for i in range(len(user_elect_price)):
        user_ew.append(int(user_elect_price[i])+int(user_water_price[i]))
        danji_ew.append(int(danji_elect_price_list[i])+int(danji_water_price_list[i]))
                    
                    
    x_train_e, x_test_e, y_train_e ,y_test_e = train_test_split(months, danji_elect_price_list, test_size=0.2, random_state=777)
    x_train_w, x_test_w, y_train_w ,y_test_w = train_test_split(months, danji_water_price_list, test_size=0.2, random_state=777)
                    
    # C가 낮을수록 일반적인 결정, 높을수록 이상치의 존재 가능성 작게 봐서 세심한 결정(너무 낮거나 높으면 과소, 과대적합될 수 있음)
    # gamma는 하나의 데이터 샘플이 영향력을 행사하는 거리 결정, 값이 클수록 작은 표준편차 가짐
    rbf_svr_e = SVR(kernel='rbf', C=5000, gamma='auto')
    rbf_svr_w = SVR(kernel='rbf', C=2000, gamma='auto')

    rbf_svr_e.fit(x_train_e, y_train_e)
    rbf_svr_w.fit(x_train_w, y_train_w)
    elect_predict = rbf_svr_e.predict(x_test_e)
    water_predict = rbf_svr_w.predict(x_test_w)

    nextMonth = [[len(months)+1]]
    # print('예상 전기료: ', round(elect_predict[0]),",",round(elect_predict[1]),","
    #       ,round(elect_predict[2]),",",round(elect_predict[3]),",",round(elect_predict[4]) )
    # print("진짜 전기료:", danji_elect_price_list[x_test_e[0][0]],",",danji_elect_price_list[x_test_e[1][0]],","
    #       ,danji_elect_price_list[x_test_e[2][0]],",",danji_elect_price_list[x_test_e[3][0]],",",danji_elect_price_list[x_test_e[4][0]],"\n")

    # print('예상 수도료: ', round(water_predict[0]),",",round(water_predict[1]),","
    #       ,round(water_predict[2]),",",round(water_predict[3]),",",round(water_predict[4]) )
    # print("진짜 수도료:", danji_water_price_list[x_test_w[0][0]],",",danji_water_price_list[x_test_w[1][0]],","
    #       ,danji_water_price_list[x_test_w[2][0]],",",danji_water_price_list[x_test_w[3][0]],",",danji_water_price_list[x_test_w[4][0]],"\n")
    
    print("\n전기료 훈련 정확도:", round(rbf_svr_e.score(x_train_e, y_train_e),2)) # 74.29
    # # print("전기료 테스트 정확도: ", r2_score(y_test_e, elect_predict)) # 74.11

    print("수도료 훈련 정확도:", round(rbf_svr_w.score(x_train_w, y_train_w),2)) # 74.29
    # # print("수도료 테스트 정확도: ", r2_score(y_test_w, water_predict))  # 74.11

    n_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-1)).strftime("%Y%m")

    danji_elect_predict = round(rbf_svr_e.predict(nextMonth)[0])
    danji_water_predict = round(rbf_svr_w.predict(nextMonth)[0])

    danji_ew_predict = danji_elect_predict+danji_water_predict

    print("우리 아파트 단지",n_month,"예상 금액(전기+수도):",danji_ew_predict,"원")
    change_rate=round(danji_ew_predict/danji_ew[-1],2)
    user_ew_predict_danji_using = round(user_ew[-1]*change_rate)
    print("아파트 평균 금액 변화율에 따른 우리 집",n_month,"예상 금액(전기+수도):",user_ew_predict_danji_using,"원")

    ew_predict_dict={}
    ew_predict_dict[n_month] = [danji_ew_predict],[user_ew_predict_danji_using] # n_month(예측날짜): 아파트 단지 예상금액, 우리집 예상 금액
    
    return n_month, danji_ew_predict, user_ew_predict_danji_using

danji_predict(29,"삼각그린타운")

## sido_code: {'대구':'27','인천':'28','광주':'29','대전':'30','울산':'31','경기':'41','강원':'42','충북':'43','충남':'44','전북':'45','전남':'46','경북':'47','경남':'48','제주':'50'}





전기료 훈련 정확도: 0.72
수도료 훈련 정확도: 0.86
우리 아파트 단지 202204 예상 금액(전기+수도): 48220 원
아파트 평균 금액 변화율에 따른 우리 집 202204 예상 금액(전기+수도): 69389 원


{'202204': ([48220], [69389])}

In [17]:
# ### k-apt api를 이용한 아파트 관리비 예측 구현(13개월 데이터로 예측)

# # Radial Basis Function(RBF)커널을 사용한 SVM(서포트 백터 머신)으로 예측 알고리즘 구현

# from sklearn.svm import SVR
# import pandas as pd
# # from pandas_datareader import data as pdr
# import matplotlib.pyplot as plt
# from sklearn.metrics import mean_squared_error

# test_elect_price=[35000,38010,28150,28800,27160,39330,37470,51389,42140,34740,36410,33390,40610] # 41280
# test_water_price=[24990,28430,29780,24390,25740,25740,29780,28430,28430,32470,27090,28430,23050] # 2574
# danji_ew=[]
# user_ew=[]
# user_elect_price=test_elect_price
# user_water_price=test_water_price
# # def danji_user_predict(user_elect_price, user_water_price):

# months=[]
# j=1
# for j in range(len(danji_elect_price_list)):
#     months.append([int(j+1)])
#     j=j+1
    
# for i in range(len(user_elect_price)):
#     user_ew.append(int(user_elect_price[i])+int(user_water_price[i]))
#     danji_ew.append(int(danji_elect_price_list[i])+int(danji_water_price_list[i]))
                    
                    
# x_train_e, x_test_e, y_train_e ,y_test_e = train_test_split(months, danji_elect_price_list, test_size=0.2, random_state=777)
# x_train_w, x_test_w, y_train_w ,y_test_w = train_test_split(months, danji_water_price_list, test_size=0.2, random_state=777)
                    
# # C가 낮을수록 일반적인 결정, 높을수록 이상치의 존재 가능성 작게 봐서 세심한 결정(너무 낮거나 높으면 과소, 과대적합될 수 있음)
# # gamma는 하나의 데이터 샘플이 영향력을 행사하는 거리 결정, 값이 클수록 작은 표준편차 가짐
# rbf_svr_e = SVR(kernel='rbf', C=5000, gamma='auto')
# rbf_svr_w = SVR(kernel='rbf', C=2000, gamma='auto')

# rbf_svr_e.fit(x_train_e, y_train_e)
# rbf_svr_w.fit(x_train_w, y_train_w)
# elect_predict = rbf_svr_e.predict(x_test_e)
# water_predict = rbf_svr_w.predict(x_test_w)

# nextMonth = [[len(months)+1]]
# # print('예상 전기료: ', round(elect_predict[0]),",",round(elect_predict[1]),","
# #       ,round(elect_predict[2]),",",round(elect_predict[3]),",",round(elect_predict[4]) )
# # print("진짜 전기료:", danji_elect_price_list[x_test_e[0][0]],",",danji_elect_price_list[x_test_e[1][0]],","
# #       ,danji_elect_price_list[x_test_e[2][0]],",",danji_elect_price_list[x_test_e[3][0]],",",danji_elect_price_list[x_test_e[4][0]],"\n")

# # print('예상 수도료: ', round(water_predict[0]),",",round(water_predict[1]),","
# #       ,round(water_predict[2]),",",round(water_predict[3]),",",round(water_predict[4]) )
# # print("진짜 수도료:", danji_water_price_list[x_test_w[0][0]],",",danji_water_price_list[x_test_w[1][0]],","
# #       ,danji_water_price_list[x_test_w[2][0]],",",danji_water_price_list[x_test_w[3][0]],",",danji_water_price_list[x_test_w[4][0]],"\n")

# print("\n전기료 훈련 정확도:", round(rbf_svr_e.score(x_train_e, y_train_e),2)) # 74.29
# # # print("전기료 테스트 정확도: ", r2_score(y_test_e, elect_predict)) # 74.11

# print("수도료 훈련 정확도:", round(rbf_svr_w.score(x_train_w, y_train_w),2)) # 74.29
# # # print("수도료 테스트 정확도: ", r2_score(y_test_w, water_predict))  # 74.11

# n_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-1)).strftime("%Y%m")

# danji_elect_predict = round(rbf_svr_e.predict(nextMonth)[0])
# danji_water_predict = round(rbf_svr_w.predict(nextMonth)[0])

# danji_ew_predict = danji_elect_predict+danji_water_predict

# print("우리 아파트 단지",n_month,"예상 금액(전기+수도):",danji_ew_predict,"원")
# change_rate=round(danji_ew_predict/danji_ew[-1],2)
# user_ew_predict_danji_using = round(user_ew[-1]*change_rate)
# print("아파트 평균 금액 변화율에 따른 우리 집",n_month,"예상 금액(전기+수도):",user_ew_predict_danji_using,"원")



In [18]:
from flask import Flask
app = Flask (__name__)

@app.route('/')
def hello_world():
    return 'Hello'

# @app.route('/gojiseoInform/<userImg>')
# def _gojiseo(userImg):
#     pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'
#     config = ('-l kor+eng --oem 3 --psm 4') 
#     img=cv2.imread(user_img)

#     def gray_scale(image):
#         return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     def deskew(image): # 숫자 이미지를 읽어 기울어진 이미지 보정
#         gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         gray = cv2.bitwise_not(gray)
#         thresh = cv2.threshold(gray, 0, 255,
#             cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
#         coords = np.column_stack(np.where(thresh > 0))
#         angle = cv2.minAreaRect(coords)[-1]
#         if angle < -45:
#             angle = -(90 + angle)
#         else:
#             angle = -angle
#         (h, w) = image.shape[:2]
#         center = (w // 2, h // 2)
#         M = cv2.getRotationMatrix2D(center, angle, 1.0)
#         rotated = cv2.warpAffine(image, M, (w, h),
#             flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)    
#         return rotated

#     deskew = deskew(img)
#     gray = gray_scale(deskew)

#     x_indivi=770; y_indivi=20; w_indivi=450; h_indivi=340 
#     x_date=480; y_date=450; w_date=220; h_date=270
#     x_price=480; y_price=670; w_price=230; h_price=270 

#     roi1 = gray[x_indivi:x_indivi+h_indivi , y_indivi:y_indivi+w_indivi] 
#     roi2 = gray[x_date:x_date+h_date , y_date:y_date+w_date]  
#     roi3 = gray[x_price:x_price+h_price, y_price:y_price+w_price]  

#     cv2.rectangle(roi1, (0,0), (w_indivi-1, h_indivi-1), (0,0,255) ,2) 
#     cv2.rectangle(roi2, (0,0), (w_date-1, h_date-1), (0,0,255) ,2) 
#     cv2.rectangle(roi3, (0,0), (w_price-1, h_price-1), (0,0,255) ,2) 

#     text_indivi=pytesseract.image_to_string(roi1,config=config)
#     text_date=pytesseract.image_to_string(roi2,config=config)
#     text_price=pytesseract.image_to_string(roi3,config=config)
    
#     start_date=text_date.find('월분')+2
#     date_l=text_date[start_date:start_date+10].strip().strip('、').strip(',').strip('.').split("-")
#     date_this=date_l[0]+date_l[1]

#     start_use_e = text_indivi.find('전기')+2
#     elec_use = int(text_indivi[start_use_e:start_use_e+4].strip())
#     start_price_e = text_indivi.find('kw')+2
#     elec_price = text_indivi[start_price_e:start_price_e+18].strip()
#     elec_price = int(elec_price.replace(",",'').strip())

#     start_use_w = text_indivi.find('수도')+2
#     water_use = int(text_indivi[start_use_w:start_use_w+3].strip())
#     start_price_w = text_indivi.find('수도')+15
#     water_price = text_indivi[start_price_w:start_price_w+15].strip()
#     water_price = int(water_price.replace(",",'').strip())

#     splits=text_price.splitlines()
#     price_this=splits[2]
#     total_price = int(price_this.replace(",",'').strip())

#     start_use_public = text_indivi.find('공용소계')+10
#     public_price = text_indivi[start_use_public:start_use_public+10].strip()
#     public_price = int(public_price.replace(",",'').strip())

#     gojiseo[date_this]=[elec_use, elec_price, water_use, water_price, public_price, total_price]    
    
#     return gojiseo



# @app.route('/piegraphInform')
# def pie_graph():
#     etc_indivi_price = total_price-(elec_price + wat_price + public_price)
#     return elec_price, wat_prece, etc_indivi_price, publi_price, total_price

# @app.route('/lastyearComparison')
# def rbf_predict():
#     months=[]
#     j=1
#     for j in range(len(test_elect_price)):
#         months.append([int(j+1)])
#         j=j+1

#     x_train_t, x_test_t, y_train_t ,y_test_t = train_test_split(months,test_total_price,test_size=0.2, random_state=777)

#     rbf_svr_t1 = SVR(kernel='rbf', C=20000, gamma=0.3)
#     rbf_svr_t1.fit(x_train_t, y_train_t)

    
#     nextMonth = [[len(months)+1]]
#     total_predict = rbf_svr_t1.predict(x_test_t)

#     rising_rate=(rbf_svr_t1.predict(nextMonth)/test_total_price[-11])*100
#     comparison=round(rising_rate[0],2)
#     # print("\n 작년 동월 금액의 약",comparison,"% 입니다.")
#     return comparison

# @app.route('/danjiCompariconPredict')
# def danji_price(sido_code, apt_name):
#     ## sido_code: {'대구':'27','인천':'28','광주':'29','대전':'30','울산':'31','경기':'41','강원':'42','충북':'43','충남':'44','전북':'45','전남':'46','경북':'47','경남':'48','제주':'50'}
#     sido,name=sido_code, apt_name
#     def apt_danji(sido,name):
#         service_dkey = 'X3P5kZCUe75RwQN0GH80YdI51JbHHwlOWkQLJiRyFIbgn/RiaVJmjOX/yFbt50x4rAhtPGt4Jlya+rIJlUefaA=='
#         url = 'http://apis.data.go.kr/1613000/AptListService2/getSidoAptList'
#         param = {'serviceKey':service_dkey, 'sidoCode':sido, 'pageNo':1, 'numOfRows':'1'}
#         r = requests.get(url,params=param)
    
#         xmlData = r.content.decode('utf-8')
#         parseData = xmltodict.parse(xmlData)
#         jData = json.loads(json.dumps(parseData))
    
#         rows=jData['response']['body']['totalCount']
    
#         pages = int(int(rows)/100)+1    
#         row_cnt=1
#         for i in range(1, pages+1):
#             params={'serviceKey':service_dkey, 'sidoCode':sido, 'pageNo':i, 'numOfRows':'100'}
        
#             response = requests.get(url, params=params)
#             xml_data = response.content.decode('utf-8')
#             parse_data = xmltodict.parse(xml_data)
        
#             ord_data = parse_data['response']['body']['items']['item']
        
#             try:
#                 jdata = json.loads(json.dumps(ord_data))
#                 input_danji_name=name
            
#                 for code_data in jdata:
#                     danji_code=code_data['kaptCode']
#                     danji_name=code_data['kaptName']
#                     row_cnt += 1
#                     if danji_name == input_danji_name:
#                         # print(code_data)
#                         return danji_code
                
#             except Exception as e:
#                 print(f'Error: {e}\n')
#                 break

#     ### api로 단지별 세대수 불러오기
#     def danji_people(code):
#         url = 'http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo'
#         params ={'serviceKey':'X3P5kZCUe75RwQN0GH80YdI51JbHHwlOWkQLJiRyFIbgn/RiaVJmjOX/yFbt50x4rAhtPGt4Jlya+rIJlUefaA==',
#          'kaptCode' : code}
#         response = requests.get(url, params=params)
#         xml_data = response.content.decode('utf-8')
#         parse_data = xmltodict.parse(xml_data)
#         ord_data = parse_data['response']['body']
#         jData = json.loads(json.dumps(ord_data))
#         jData = jData['item']
#         return jData['kaptdaCnt']

#     ### api로 단지별 전기 사용량 불러오기
#     def danji_elect(code,date):
#         url = 'http://apis.data.go.kr/1611000/AptIndvdlzManageCostService/getHsmpElectricityCostInfo'
#         params ={'serviceKey':'X3P5kZCUe75RwQN0GH80YdI51JbHHwlOWkQLJiRyFIbgn/RiaVJmjOX/yFbt50x4rAhtPGt4Jlya+rIJlUefaA==',
#          'kaptCode' : code, 'searchDate' : str(date) }
#         response = requests.get(url, params=params)
#         xml_data = response.content.decode('utf-8')
#         parse_data = xmltodict.parse(xml_data)
#         ord_data = parse_data['response']['body']
#         jData = json.loads(json.dumps(ord_data))
#         jData = jData['item']
#         return jData['electP']

#     ### api로 단지별 수도 사용량 불러오기
#     def danji_water(code,date):
#         url = 'http://apis.data.go.kr/1611000/AptIndvdlzManageCostService/getHsmpWaterCostInfo'
#         params ={'serviceKey':'X3P5kZCUe75RwQN0GH80YdI51JbHHwlOWkQLJiRyFIbgn/RiaVJmjOX/yFbt50x4rAhtPGt4Jlya+rIJlUefaA==',
#          'kaptCode' : code, 'searchDate' : str(date) }
#         response = requests.get(url, params=params)
#         xml_data = response.content.decode('utf-8')
#         parse_data = xmltodict.parse(xml_data)
#         ord_data = parse_data['response']['body']
#         jData = json.loads(json.dumps(ord_data))
#         jData = jData['item']
#         return jData['waterCoolP']

#     danji_price_dict={}
#     danji_elect_price_list=[]
#     danji_water_price_list=[]

#     months_list=[]
#     present_date = datetime.today().strftime("%Y%m")

#     # 단지 평균 전기료, 수도료 조회 (13개월 16-3)
#     for i in range(2,27): # 현재 달로부터 3개월 전부터 조회가능
#         ago_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-(28-i))).strftime("%Y%m")
#         danji_code = apt_danji(sido_code,apt_name)
        
#         electP = danji_elect(danji_code, ago_month) # 우리 단지 총액
#         waterP = danji_water(danji_code, ago_month) # 우리 단지 총액

#         danji_elect_price = round(int(electP)/int(danji_people(danji_code))) # 우리 단지 평균
#         danji_water_price = round(int(waterP)/int(danji_people(danji_code)))
#         danji_elect_price_list.append(danji_elect_price)
#         danji_water_price_list.append(danji_water_price)
    
#         danji_price_dict[ago_month]= [danji_elect_price, danji_water_price] # 날짜 : 전기료, 수도료
#         months_list.append(ago_month)
    
    
    
#     test_elect_price=[35000,38010,28150,28800,27160,39330,37470,51389,42140,34740,36410,33390,40610] # 41280
#     test_water_price=[24990,28430,29780,24390,25740,25740,29780,28430,28430,32470,27090,28430,23050] # 2574
#     danji_ew=[]
#     user_ew=[]
#     user_elect_price=test_elect_price
#     user_water_price=test_water_price

#     months=[]
#     j=1
#     for j in range(len(danji_elect_price_list)):
#         months.append([int(j+1)])
#         j=j+1
    
#     for i in range(len(user_elect_price)):
#         user_ew.append(int(user_elect_price[i])+int(user_water_price[i]))
#         danji_ew.append(int(danji_elect_price_list[i])+int(danji_water_price_list[i]))
                    
                    
#     x_train_e, x_test_e, y_train_e ,y_test_e = train_test_split(months, danji_elect_price_list, test_size=0.2, random_state=777)
#     x_train_w, x_test_w, y_train_w ,y_test_w = train_test_split(months, danji_water_price_list, test_size=0.2, random_state=777)

#     rbf_svr_e = SVR(kernel='rbf', C=5000, gamma='auto')
#     rbf_svr_w = SVR(kernel='rbf', C=2000, gamma='auto')

#     rbf_svr_e.fit(x_train_e, y_train_e)
#     rbf_svr_w.fit(x_train_w, y_train_w)
#     elect_predict = rbf_svr_e.predict(x_test_e)
#     water_predict = rbf_svr_w.predict(x_test_w)

#     nextMonth = [[len(months)+1]]
#     n_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-1)).strftime("%Y%m")

#     danji_elect_predict = round(rbf_svr_e.predict(nextMonth)[0])
#     danji_water_predict = round(rbf_svr_w.predict(nextMonth)[0])

#     danji_ew_predict = danji_elect_predict+danji_water_predict

#     change_rate=round(danji_ew_predict/danji_ew[-1],2)
#     user_ew_predict_danji_using = round(user_ew[-1]*change_rate)

#     ew_predict_dict={}
#     ew_predict_dict[n_month] = [danji_ew_predict],[user_ew_predict_danji_using] # n_month(예측날짜): 아파트 단지 예상금액, 우리집 예상 금액
#     return ew_predict_dict


if __name__=="__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
